In [18]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import sys
import glob
import pandas as pd
import os
import seaborn as sns

from tqdm import tqdm
from statsmodels.distributions.empirical_distribution import ECDF
from collections import defaultdict
import pickle
import re
import json
from pathlib import Path


from open_spiel.python.algorithms.exploitability import nash_conv, best_response
from open_spiel.python.examples.ubc_plotting_utils import *


import bokeh
from bokeh.layouts import row, column
from bokeh.plotting import figure, show, output_file, save
from bokeh.io import output_notebook
from bokeh.models import HoverTool, ColumnDataSource, ColorBar, LogColorMapper, LinearColorMapper
from bokeh.transform import linear_cmap, log_cmap
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from auctions.webutils import *

from sklearn.model_selection import ParameterGrid
import json
import copy

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
base_name = '/apps/open_spiel/configs/sats_3regions_3licenses_5types.json'
with open(base_name, 'r') as f:
    base_config = json.load(f)
    


In [20]:
choices = {
    'information_policy': ['show_demand', 'hide_demand'],
    'pricing_bonus': [0.0, 0.05, 0.25],
}

grid = ParameterGrid(choices)

In [32]:
def write_config(base_config, params, base_name):
    config = copy.deepcopy(base_config)
    config['information_policy'] = params['information_policy']
    for player in base_config['players']:
        for player_type in range(len(player['type'])):
            player['type'][player_type]['pricing_bonus'] = params['pricing_bonus']
    
    extension = '_hide' if params['information_policy'] == 'hide_demand' else ''
    if params.get('pricing_bonus', 0) > 0:
        pretty_pricing = int(100 * params["pricing_bonus"])
        extension += f'_spiteful_{pretty_pricing}'
    base_name = base_name.replace('.json','')
    with open(f'{base_name}{extension}.json', 'w') as f:
        json.dump(config, f, indent=4)

In [33]:
for config in grid:
    write_config(base_config, config, base_name)